<a href="https://colab.research.google.com/github/gideonoludeyi/cosc5p70/blob/main/notebooks/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Development
#### Predict Students' Dropout and Academic Success

## Setup

Includes installation of packages, importing modules, and downloading dataset used for model development.

In [1]:
# Install package dependencies
!pip install "ucimlrepo" "pandas" "numpy" "matplotlib" "torch" "scikit-learn"

In [2]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim
import random
import time
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight # Used to counter bias in an imbalanced dataset

The `ucimlrepo` is used to download the training dataset by Realinho et al. (2021)<sup>1</sup> hosted on the UC Irvine Machine Learning Repository.

[1] Realinho, V., Vieira Martins, M., Machado, J., & Baptista, L. (2021). Predict Students' Dropout and Academic Success [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5MC89.

[2] M.V.Martins, D. Tolledo, J. Machado, L. M.T. Baptista, V.Realinho. (2021) "Early prediction of student’s performance in higher education: a case study" Trends and Applications in Information Systems and Technologies, vol.1, in Advances in Intelligent Systems and Computing series. Springer. DOI: 10.1007/978-3-030-72657-7_16


In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset - https://archive.ics.uci.edu/dataset/697
repo = fetch_ucirepo(id=697)

# data (as pandas dataframes)
X = repo.data.features

X_norm = nn.functional.normalize(torch.from_numpy(X.values), p=2, dim=1)
y = repo.data.targets['Target']

X = pd.DataFrame(X_norm.numpy(), columns=X.columns)

# metadata
# print(repo.metadata)

# variable information
# print(repo.variables)

A random seed is used to guarantee reproducible results of stochastic processes (e.g., generating random numbers).

In [4]:
# Fixing the random seed to guarantee deterministic results
def set_seed(seed):
    import os
    import random
    import numpy as np
    import torch
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

#seed = 123456789
seed = int(time.time() * 1000) % (2**32)

set_seed(seed)
rng = torch.Generator().manual_seed(seed)

## Data Preprocessing

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 36 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital Status                                  4424 non-null   float64
 1   Application mode                                4424 non-null   float64
 2   Application order                               4424 non-null   float64
 3   Course                                          4424 non-null   float64
 4   Daytime/evening attendance                      4424 non-null   float64
 5   Previous qualification                          4424 non-null   float64
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   float64
 8   Mother's qualification                          4424 non-null   float64
 9   Father's qualification                   

In [6]:
# Remove the gender and nationality features from the dataset
X = X.drop(columns=['Gender', 'Nacionality'])

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 34 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital Status                                  4424 non-null   float64
 1   Application mode                                4424 non-null   float64
 2   Application order                               4424 non-null   float64
 3   Course                                          4424 non-null   float64
 4   Daytime/evening attendance                      4424 non-null   float64
 5   Previous qualification                          4424 non-null   float64
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Mother's qualification                          4424 non-null   float64
 8   Father's qualification                          4424 non-null   float64
 9   Mother's occupation                      

In [8]:
X.describe()

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,...,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,0.000324,0.004356,0.000477,0.980495,0.000288,0.001083,0.040599,0.005127,0.006014,0.002437,...,0.000015,0.000283,0.001153,0.001482,0.000851,0.001590,0.000025,0.003452,0.000351,0.000038
std,0.000860,0.017999,0.001556,0.086786,0.000838,0.007689,0.113942,0.018795,0.021118,0.011086,...,0.000075,0.002919,0.004894,0.006274,0.004149,0.005018,0.000291,0.009820,0.001506,0.002068
min,0.000100,0.000100,0.000000,0.151658,0.000000,0.000100,0.009823,0.000100,0.000100,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000760,-0.003446,-0.016213
25%,0.000105,0.000109,0.000105,0.999746,0.000103,0.000105,0.013309,0.000304,0.000330,0.000421,...,0.000000,0.000000,0.000550,0.000656,0.000219,0.001137,0.000000,0.000989,0.000032,-0.000184
50%,0.000108,0.001837,0.000110,0.999785,0.000108,0.000108,0.014286,0.002077,0.003120,0.000683,...,0.000000,0.000000,0.000649,0.000842,0.000541,0.001314,0.000000,0.001224,0.000143,0.000035
75%,0.000110,0.004263,0.000220,0.999811,0.000110,0.000111,0.015260,0.003999,0.004004,0.000974,...,0.000000,0.000000,0.000748,0.001093,0.000660,0.001443,0.000000,0.001522,0.000284,0.000194
max,0.009554,0.262869,0.023949,0.999885,0.005837,0.204277,0.745404,0.221806,0.215969,0.464207,...,0.001321,0.064730,0.081711,0.109082,0.072135,0.071404,0.013787,0.094559,0.014809,0.019657


In [9]:
# Observation
# There are three labels: 'Dropout', 'Enrolled', and 'Graduate'
# The dataset is imbalanced towards instances of 'Graduate'.
#   May lead to bias in the model
y.value_counts()

,count
Target,
Graduate,2209
Dropout,1421
Enrolled,794


In [10]:
# View a few data samples
X.sample(10, random_state=seed)

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
1945,0.000105,0.000105,0.000421,0.999806,0.000105,0.000105,0.013576,0.003894,0.003894,0.000947,...,0.000000,0.000000,0.000842,0.000842,0.000842,0.001575,0.00000,0.001305,0.000053,0.000188
2788,0.000103,0.004032,0.000103,0.999773,0.000103,0.000103,0.015302,0.001964,0.001964,0.000724,...,0.000000,0.000000,0.000620,0.000931,0.000000,0.000000,0.00000,0.001117,0.000145,0.000180
2732,0.000105,0.000105,0.000105,0.999811,0.000105,0.000105,0.013787,0.002000,0.002000,0.000421,...,0.000000,0.000000,0.000842,0.000842,0.000737,0.001475,0.00000,0.001337,0.000389,-0.000179
119,0.000109,0.000109,0.000109,0.999807,0.000109,0.000109,0.013882,0.000219,0.002077,0.000437,...,0.000000,0.000000,0.000547,0.000547,0.000547,0.001530,0.00000,0.001213,0.000066,0.000221
977,0.000220,0.000110,0.000110,0.999425,0.000110,0.000110,0.015291,0.002090,0.002090,0.021122,...,0.000000,0.000000,0.000550,0.000770,0.000440,0.001458,0.00011,0.001188,0.000154,0.000191
2536,0.000110,0.004709,0.000110,0.999852,0.000110,0.000110,0.010951,0.000329,0.000438,0.000329,...,0.000219,0.000000,0.000548,0.000767,0.000548,0.001658,0.00000,0.001029,-0.000088,-0.000342
1860,0.000219,0.004276,0.000110,0.999794,0.000110,0.000110,0.012060,0.002083,0.002083,0.000000,...,0.000000,0.000000,0.000548,0.000548,0.000000,0.000000,0.00000,0.000833,0.000285,0.000035
1058,0.000103,0.004032,0.000103,0.999835,0.000103,0.001241,0.013762,0.000103,0.001965,0.000103,...,0.000000,0.000000,0.000517,0.000517,0.000414,0.001163,0.00000,0.000972,-0.000083,-0.000323
3889,0.000105,0.000105,0.000209,0.999813,0.000105,0.000105,0.013601,0.000105,0.003976,0.000314,...,0.000000,0.000000,0.000837,0.000837,0.000837,0.001478,0.00000,0.001622,0.000293,-0.000425
2492,0.000125,0.006362,0.000125,0.999725,0.000000,0.000125,0.014970,0.000125,0.000125,0.000499,...,0.000000,0.000624,0.001247,0.001996,0.000873,0.001301,0.00000,0.001547,0.000062,0.000223


Data splitting

The dataset is split into training and testing datasets for training the model and evaluating the model's performance respectively.

We use 80% for training and 20% for testing.

In [11]:
# Data splitting - training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=seed)

print(f"Train set size: {len(y_train)} ({len(y_train) / len(y):.2f})")
print(f"Test  set size: {len(y_test)}  ({len(y_test) / len(y):.2f})")

Train set size: 3539 (0.80)
Test  set size: 885  (0.20)


Standardize the input features to a mean of `0` and standard deviation of `1`.

$$
z = \frac{x - \mu}{\sigma}
$$

where $x$ is the input feature, $\mu$ is the mean, and $\sigma$ is the standard deviation of the dataset.

In [12]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [13]:
# Save the mean and standard deviation values for preprocessing future inputs.
mean.to_csv('mean.csv', index=True)
std.to_csv('std.csv', index=True)

In [14]:
def label_to_index(label):
    """
    Convert a label to an appropriate index.
    Ensures that the label can be used in a neural network.
    """
    map = {'Dropout': 0, 'Enrolled': 1, 'Graduate': 2}
    return map[label]

def index_to_label(index):
    """
    Utility function to convert an index to a label.
    """
    map = {0: 'Dropout', 1: 'Enrolled', 2: 'Graduate'}
    return map[index]

In [15]:
# Encode target labels as integers
y_train = y_train.apply(label_to_index)
y_test = y_test.apply(label_to_index)

## Model development and training

In [16]:
class FocalLoss(nn.Module):
    """
    An implementation of the focal loss function for multi-label classification by Lin et at. (2017).

    Lin, T., Goyal, P., Girshick, R.B., He, K., & Dollár, P. (2017). Focal Loss for Dense Object Detection.
    IEEE Transactions on Pattern Analysis and Machine Intelligence, 42, 318-327.
    https://doi.org/10.48550/arXiv.1708.02002
    """
    def __init__(self, alpha=1, gamma=2, weights=None):
        """
        Args:
            weights: A tensor of weights for each class.
                Used to rebalance the output of the loss function for imbalanced labels.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weights = weights

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss(weight=self.weights)(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss

In [17]:
class SupervisedDataset(Dataset):
    """
    A generic `Dataset` container for supervised data
    with a set of features and accompanying target labels.
    """
    def __init__(self, X, y, transform=None, target_transform=None):
        """
        Args:
            X: a tensor representing the features
            y: a tensor representing the labels
            transform: (optional) a function to process the features
            target_transform: (optional) a function to process the labels
        """
        self.X = np.asarray(X)
        self.y = np.asarray(y)
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        features = self.X[index]
        label = self.y[index]
        if self.transform is not None:
            features = self.transform(features)
        if self.target_transform is not None:
           label = self.target_transform(label)
        return features, label

    def __len__(self):
        return len(self.y)

In [18]:
def train_step(dataloader, model, loss_fn, optimizer):
    """
    Applies a training procedure for a neural network model

    Args:
        dataloader: a `DataLoader` instance for retrieving the data used for training
        model: a `nn.Module` instance representing the neural network
        loss_fn: a `nn.Module` instance representing the loss function
        optimizer: a `torch.optim.Optimizer` algorithm to optimize the model's parameters
    """
    size = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Convert X and y to float tensors to match model requirements
        X, y = X.clone().detach().float(), y.clone().detach().long()

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % (batch_size // 4) == 0:
            loss = loss.item()
            current = batch * dataloader.batch_size + len(X)
            #print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_step(dataloader, model, loss_fn):
    """
    Evaluate the model's performance on the given dataset

    Args:
        dataloader: a `DataLoader` instance for retrieving the data used for evaluation
        model: a `nn.Module` instance representing the neural network
        loss_fn: a `nn.Module` instance representing the loss function
    Return:
        a tuple container the average loss and accuracy of the model on the given dataset
    """
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.clone().detach().float(), y.clone().detach().long()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    return test_loss, correct

We define a Feed-forward Artificial Neural Network model with 3 hidden layers and an output layer with 3 output nodes each representing the likelihood that a given input feature corresponds to 'Graduate', 'Enrolled', and 'Dropout'.

In [19]:
input_dimensions = np.shape(X_train)[1]

print(f"Input dimensions: {input_dimensions}")

model = nn.Sequential(
    nn.Linear(input_dimensions, 256),  # First hidden layer
    nn.BatchNorm1d(256),               # Normalize activations
    nn.ReLU(),
    nn.Dropout(0.3),                   # Regularization
    nn.Linear(256, 128),                # Second hidden layer
    nn.BatchNorm1d(128),                # Normalize activations
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),               # Third hidden layer
    nn.BatchNorm1d(64),                # Normalize activations
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 3)                   # Output layer
)

print(model)

Input dimensions: 34
Sequential(
  (0): Linear(in_features=34, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.3, inplace=False)
  (8): Linear(in_features=128, out_features=64, bias=True)
  (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Dropout(p=0.3, inplace=False)
  (12): Linear(in_features=64, out_features=3, bias=True)
)


In [20]:
# Hyperparameter definitions
learning_rate = 0.001
epochs = 1000
momentum=0.9
batch_size = 64

In [21]:
# Loss function
#  assign weights labels due to imbalance in labels
#  e.g., there are more 'Graduate' instances than 'Enrolled' and 'Dropout'
#        so 'Graduate' label is assigned a lower weight in term of loss
classes_train = np.unique(y_train)
loss_weights = compute_class_weight('balanced', classes=classes_train, y=y_train)
loss_weights = torch.tensor(loss_weights, dtype=torch.float32)
print(f"Classes:      {classes_train}")
print(f"Loss Weights: {loss_weights}")

loss_fn = FocalLoss(alpha=1, gamma=2, weights=loss_weights)

Classes:      [0 1 2]
Loss Weights: tensor([1.0375, 1.8577, 0.6676])


In [22]:
# Stochastic Gradient Descent algorithm for training the model
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

Apply the training procedure over a certain number of epochs.

Display the outputs every 50 epochs.

In [ ]:
train_data = SupervisedDataset(X_train, y_train)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, generator=rng)

test_data = SupervisedDataset(X_test, y_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, generator=rng)

best_accuracy = float('-inf')

for epoch in range(epochs):
    # Training step
    train_step(train_dataloader, model, loss_fn, optimizer)
    # Validation step
    test_loss, accuracy = test_step(test_dataloader, model, loss_fn)

    if epoch % 50 == 0:
      print(f"Epoch {epoch + 1}\n-------------------------------")
      print(f"Test Error: \n test_loss: {(100*accuracy):>0.1f}%, avg.loss: {test_loss:>8f} \n")

    # Below simply stops the training process if no improvement has happened for a while
    # This way, if we oscillate, we only take the best weights before decreasing
    # Check if validation loss improved
    if accuracy > best_accuracy:
        best_accuracy = accuracy  # Update the best validation loss
        # Save the model with the best performance
        torch.save(model.state_dict(), "best_model.pth")
        if epoch % 50 == 0:
          print(f"Validation loss improved to {(100*best_accuracy):>0.1f}%")
    else:
        if epoch % 50 == 0:
          print(f"No improvement. Best accuracy is still: {(100*best_accuracy):>0.1f}%")

Epoch 1
-------------------------------
Test Error: 
 test_loss: 50.1%, avg.loss: 0.492254 

Validation loss improved to 50.1%
Epoch 51
-------------------------------
Test Error: 
 test_loss: 72.0%, avg.loss: 0.240875 

No improvement. Best accuracy is still: 72.8%
Epoch 101
-------------------------------
Test Error: 
 test_loss: 71.0%, avg.loss: 0.224601 

No improvement. Best accuracy is still: 73.0%
Epoch 151
-------------------------------
Test Error: 
 test_loss: 70.6%, avg.loss: 0.222911 

No improvement. Best accuracy is still: 73.6%
Epoch 201
-------------------------------
Test Error: 
 test_loss: 71.3%, avg.loss: 0.213144 

No improvement. Best accuracy is still: 73.6%
Epoch 251
-------------------------------
Test Error: 
 test_loss: 71.9%, avg.loss: 0.218097 

No improvement. Best accuracy is still: 73.6%
Epoch 301
-------------------------------
Test Error: 
 test_loss: 73.0%, avg.loss: 0.212927 

No improvement. Best accuracy is still: 73.6%


## Model evaluation

In [ ]:
def predict(model, X):
    """
    Applies the model some data instances to produce the model's prediction.

    Returns:
        a numpy array of predictions as indices
    """
    y_pred = []
    with torch.no_grad():
        X = torch.tensor(X, dtype=torch.float32)
        pred = model(X)
        y_pred = pred.argmax(1)
    return np.asarray(y_pred)

In [ ]:
def get_result(model, features_df, y_true):
    """
    Evaluate the model on the given features `pandas.DataFrame`,
        and generates a `pandas.DataFrame` of the prediction results "y_pred"
        as well as the actual target values "y_true".

    Returns:
        a `pandas.DataFrame` of the prediction results.
    """
    X = features_df.values
    y_pred = predict(model, X)
    result_df = features_df.copy()
    result_df['y_true'] = y_true
    result_df['y_true'] = result_df['y_true'].apply(index_to_label)
    result_df['y_pred'] = y_pred
    result_df['y_pred'] = result_df['y_pred'].apply(index_to_label)
    return result_df

In [ ]:
# Generate and save model predictions on the testing dataset
test_result_df = get_result(model, X_test, y_test.values)
test_result_df.to_csv('test_result.csv', index=True)
test_result_df.head()

In [ ]:
# This calculates Predictive parity according to the formula
# PP = (number of true prediction)/(total number) per group
# The result slightly not fair to group 1 ()

predictive_parity = test_result_df.groupby(test_result_df['International']).apply(
    lambda group: (group['y_true'] == group['y_pred']).mean()
).rename("Predictive Parity")

predictive_parity

In [ ]:
# Plot precision, recall, and f1-score metrics
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

def plot_precision_recall_fscore(y_true, y_pred, ax=None):
    if ax is None:
        fig, ax = plt.subplots(layout='constrained')

    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    labels = ("Graduate", "Enrolled", "Dropout")
    penguin_means = {
        'Precision': (precision[0], precision[1], precision[2]),
        'Recall': (recall[0], recall[1], recall[2]),
        'F-score': (fscore[0], fscore[1], fscore[2]),
    }

    x = np.arange(len(labels))  # the label locations
    width = 0.25  # the width of the bars
    multiplier = 0

    for attribute, measurement in penguin_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurement, width, label=attribute)
        ax.bar_label(rects, padding=3, fmt='%.2f')
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Score')
    ax.set_title('Precision, Recall, F1-Score by labels')
    ax.set_xticks(x + width, labels)
    ax.legend(loc='upper left', ncols=3)
    ax.set_ylim(0, 1)

    return ax

plot_precision_recall_fscore(test_result_df['y_true'].values, test_result_df['y_pred'].values)
plt.show()

## Save model

In [ ]:
# Export the model's internal parameters
torch.save(model.state_dict(), "model.pth")
from google.colab import files
files.download("model.pth")